# Reto 3: El problema final

Este reto consiste en predecir si una imagen contiene o no signos de retinopatía diabética (DR).
Para ello se proporciona un conjunto de datos que contiene características extraídas de imágenes.
Las características extraídas son:
+ [0] Evaluación de la calidad de la imagen , donde 0 = mala calidad 1 = calidad suficiente.
+ [1] El resultado binario de la evaluación previa, donde 1 indica una anormalidad retiniana severa y 0 su falta.
+ [2-7] Los resultados de la detección de microaneurismas (MA). Cada valor de característica representa el número de MAs encontrados en los niveles de confianza alfa = 0.5, ... , 1, respectivamente.
+ [8-15] contienen la misma información que [2-7] para los exudados. <br>
Sin embargo, ya que los exudados están representados por un conjunto de puntos en lugar del número de
píxeles de las lesiones, estas características se normalizan dividiendo los número de lesiones por  el diámetro de la ROI para compensar las diferentes tamaños de las imágenes.
+ [16] La distancia euclidiana del centro dela mácula y el centro del disco óptico para proporcionar información importante con respecto a la condición del paciente. Esta característica también se normaliza con el diámetro de la ROI
+ [17] El diámetro del disco óptico.
+ [18] El resultado binario de la clasificación basada en AM/FM.
+ [19] Etiqueta de clase. 1 = contiene signos de DR

El conjunto de datos, *retinopatia_reto3.csv*, consiste en 592 muestras de imágenes; cada uno de ellos representado por 19 características.

Para evaluar las propuestas se utilizará un conjunto de datos que se mantendrá oculto hasta después de la entrega

### Requisitos
+ **Se debe utilizar algún tipo de reducción de la dimensión o aprendizaje de variedades**
+ A continuación se debe entrenar un clasificador o combinación de clasificadores con las características transformadas según el requisito anterior.
+ Se debe entregar un cuaderno Jupyter con el nombre de los participantes.<br>
  *Por ejemplo*:   **Cuesta_Hinton.ipynb**
+ El cuaderno entregado debe seguir la estructura y reglas de este cuaderno

### Competición
+ Todos los cuadernos entregados se subirán al repo de GitHub y se ejecutarán en Binder, donde ya estará en conjunto de test que estaba oculto.
+ El resultado que se obtenga será la puntuación del reto.
+ **Importante** Es muy fácil asegurarte de que tu código funciona:
    1. Agrupa todo tu código en una única celda
    2. Copialo en el cuaderno del reto que hay en Binder
    3. Ejecuta el cuaderno 
    
### Plazo: lunes 16 de nov. a las 6 am.
Es decir, incluye toda la noche del domingo 15 de nov.
 
> "The final problem", A. C. Doyle (Strand Magazine, diciembre 1983), 
es el relato corto en el que Sherlock Holmes se enfrenta al Prof. Moriarty en las cataratas Reichenbach.

---
    [ES] Código de Alfredo Cuesta Infante para 'Reconocimiento de Patrones'
       @ Master Universitario en Visión Artificial, 2020, URJC (España)
    [EN] Code by Alfredo Cuesta-Infante for 'Pattern Recognition'
       @ Master of Computer Vision, 2020, URJC (Spain)

    alfredo.cuesta@urjc.es

In [ ]:
# NO TOCAR ESTA CELDA
# Conjunto distribuido para el reto

Challange_filename = '../../Datasets/retinopatia_reto3.csv'

In [ ]:
# NO TOCAR ESTA CELDA
# El conjunto de test cambiará una vez se cierre la entrega
# Ahora mismo es el mismo conjunto que el de entrenamiento

Test_filename = '../../Datasets/retinopatia_test.csv' #<-- este nombre cambiará después del plazo de entrega

In [ ]:
# NO TOCAR ESTA CELDA

#-[1]. Load data from CSV and put all in a single dataframe 'FullSet'

import numpy  as np
import pandas as pd
from matplotlib import pyplot as plt
import sys
sys.path.append('../../MyUtils/')
import MyUtils as my
seed = 1234 #<- random generator seed (comment to get randomness)

#-[2]. Load data from CSV and put all in a single dataframe 'FullSet'

FullSet = pd.read_csv(Challange_filename, header=0)
FullX = FullSet.iloc[:,:-1]
FullY = FullSet.iloc[:,-1]

<table style="width:100%;"> 
 <tr style='background:lime'>
  <td style="text-align:left">
      <h2>Tu código debe empezar a partir de aquí y puede tener tantas celdas como quieras</h2>
      <p> Si quieres, puedes borrar (o convertir en RawNBConvert) las celdas de ejemplo
      <h3>Importante:</h3>
      <p>Tu código debe producir las siguientes variables: </p>
      <p> $\quad \bullet$ <b>clf:</b> el clasificador final con el que se realizará el test<br>
       $\quad \bullet$ <b>X_test:</b> el conjunto de test listo para ser usado por el método <b>predict</b><br>
       $\quad \bullet$ <b>Y_test:</b> es el vector de etiquetas del conjunto de X_test listo para ser usado por el método <b>confusion_matrix</b>
      </p>
  </td>
 </tr>
</table>

# Pasos realizados:
Todos estos pasos se han basado en una exploración aleatoria de los hiperparámetros que emplea cada algoritmo. Para ello, se ha realizado la división de los datos en 5 conjuntos distintos, cada uno de ellos con un 10% como subconjunto de test.

El criterio empleado en la elección de candidatos ha sido la mejor media obtenida con los 5 conjuntos de entrenamiento.

## 1. Exploración amplia
En esta fase se ha tratado de hacer un cribado de los algoritmos de reducción de dimensiones y clasificación.
### 1.1. Reductores de dimensión
Se han explorado los siguientes algoritmos: PCA, KernelPCA, LLE y LDA; y como algoritmo de clasificación el SVC(C=1, g=1).

Los resultados obtenidos en esta fase ha permitido descartar LDA por sus malos resultados (<70% acierto de media) y fijar el resto en: PCA(n=10-18), KernelPCA(n=9-18) y LLE(n=17-18).

### 1.2. Algoritmos de clasificación
Los precandidatos han sido: SVC(poly, rbf), NuSVC, Árboles de Decision(DT), Naive-Bayes(NBC), Discriminador Cuadrático(QDA), mézclas de Gausianas(GM) y Random Forest(RFC).

De todos esto se han seleccionado como candidatos aquellos que superaban el 72% de aciertos: SVC y NuSVC.

## 2. Exploración fina
Esta fase se ha centrado en la exploración de hiperparámetros (tanto a reductores como clasificadores), cada algoritmo incorporaba un valor aleatorio con un rango que se ha ido reduciendo según se obtenían mejores resultados.

Los resultados obtenidos han sido:
+ Reductores de dimensión:
    - PCA: 18.
    - KernelPCA: 18.
    - LLE: 18.
+ Clasificadores:
    - SVC(poly=4, C=22424, g=0.07).
    - SVC(rbf, C=24851, g=0.02).
    - NuSVC(g=3412.87).
    
## 3. Combinación con ensambladores
Se ha probado a combinar todos estos resultados con Voting y Bagging como algoritmos de ensamblado.
### 3.1 Voting
Incorporando a este ensamblador: 1xSVC(poly), 3xSVC(rbf), 1xNuSVC, 1xDT. Se ha vuelto a realizar una exploración de parámetros donde como media máxima se ha obtenido un 77.29% de acierto.
+ Reductores de dimensión (LLE y KernelPCA descartado):
    - PCA: 18.
+ Clasificadores (valores truncados a 2-3 decimales):
    - SVC_0(poly=4, C=22493.15, g=0.015).
    - SVC_1(rbf, C=5681.39, g=0.001).
    - SVC_2(rbf, C=24892.62, g=0.025).
    - SVC_3(rbf, C=13151.53, g=0.219).
    - NuSVC(g=3482.99).
    - DT(deep=5).
Con Voting('soft') los porcentajes siempre eran inferiores.

### 3.2 Bagging
Se ha probado con los clasificadores anteriores (parámetros aleatorios), pero ninguno conseguía superar el 63% de aciertos.

## 4. Selección final
Se ha optado por escoger el algoritmo que mejor media tenía.
+ Reductor de dimensión: PCA(16).
+ Clasificador Voting ('hard'):
    - SVC(C=20467.967046728358, degree=4, gamma=0.019547226236945454, kernel='poly', random_state=1234)
    - SVC(C=4573.7912268319415, gamma=0.03614477563143089, random_state=1234)
    - SVC(C=16879.909099563494, gamma=0.014195152228122867, random_state=1234)
    - SVC(C=7075.387287570611, gamma=0.28699813069319374, random_state=1234)
    - NuSVC(gamma=0.43917251479387664, random_state=1234)
    - DT(max_depth=7)

In [ ]:
nombres = ["Javier Albaráñez Martínez"]

In [ ]:
# Imports
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC, NuSVC
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.ensemble import VotingClassifier

# Data training processing
scaler = MinMaxScaler()
pca = PCA(n_components=18)

X_ = scaler.fit_transform(FullX)
Y = FullY.values.ravel() 
pca.fit(X_, Y)
X = pca.transform(X_)

# Data test processing
FullSet = pd.read_csv(Test_filename, header=0)
TestX_ = FullSet.iloc[:,:-1]
TestY_ = FullSet.iloc[:,-1]
X_test = pca.transform(scaler.transform(TestX_))
Y_test = TestY_.values.ravel() 

# Classifier
svm_clf = SVC(C=20467.967046728358, degree=4, gamma=0.019547226236945454, kernel='poly', random_state=seed)
svm_clf_1 = SVC(C=4573.7912268319415, gamma=0.03614477563143089, kernel='rbf', random_state=seed)
svm_clf_2 = SVC(C=16879.909099563494, gamma=0.014195152228122867, kernel='rbf', random_state=seed)
svm_clf_3 = SVC(C=7075.387287570611, gamma=0.28699813069319374, kernel='rbf', random_state=seed)
nu_svm_clf = NuSVC(gamma=0.43917251479387664, random_state=1234)
tree_clf = DT(max_depth=7, random_state=1234)
clf = VotingClassifier(estimators=[('svc', svm_clf),  ('svc1', svm_clf_1),  ('svc2', svm_clf_2), 
                                          ('svc3', svm_clf_3), ('nusvc', nu_svm_clf), ('tree',tree_clf)], voting='hard')
clf.fit(X,Y)

<table style="width:100%;"> 
 <tr style='background:pink'>
  <td style="text-align:left">
      <h2>A partir de aquí ya no se pueden modificar las celdas</h2>
          <h3>Comprueba que:</h3>
          <p> $\quad \bullet$ tu clasificador está almacenado en la variable <b>clf</b><br>
              $\quad \bullet$ tienes el conjunto de test correctamente almacenado en la variable <b>X_test</b><br>
              $\quad \bullet$ tienes las etiquetas del conjunto de test correctamente almacenadas en la variable <b>Y_test</b><br>
          </p>
      
  </td>
 </tr>
</table>

## Test

In [ ]:
# NO TOCAR ESTA CELDA

from sklearn.metrics import confusion_matrix

Y_hat = clf.predict(X_test)
conf_mat = confusion_matrix(Y_test , Y_hat)
N_success  = np.trace(conf_mat)
N_fails = Y_test.shape[0]-N_success
#-------------------------------
print (nombres,"\n")
print("Confusion matrix:\n")
print(conf_mat,"\n")
print("Outcome:\n")
strlog = "  :) HIT  = %d, (%0.2f%%)"%(N_success, 100*N_success/(N_success+N_fails))
print(strlog)
strlog = "  :( FAIL = %d, (%0.2f%%)"%(N_fails, 100*N_fails/(N_success+N_fails))
print(strlog)